<a href="https://colab.research.google.com/github/120Davies/DS-Unit-1-Sprint-1-Dealing-With-Data/blob/master/module1-afirstlookatdata/LS_DS_111_A_First_Look_at_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science - A First Look at Data



## Lecture - let's explore Python DS libraries and examples!

The Python Data Science ecosystem is huge. You've seen some of the big pieces - pandas, scikit-learn, matplotlib. What parts do you want to see more of?

In [2]:
# TODO - we'll be doing this live, taking requests
# and reproducing what it is to look up and learn things

# This is a change to the file.
2+2

4

In [0]:
myName= "McKay"

1) Fork the repo to your github account
2) go to colab.research.google.com and click on the github tab or go tot colab.resarech.google.com/github
4) work on the assignment and make changes
5) save your work back to github by going to `file`>>`Save a copy in Github...`


## Assignment - now it's your turn

Pick at least one Python DS library, and using documentation/examples reproduce in this notebook something cool. It's OK if you don't fully understand it or get it 100% working, but do put in effort and look things up.

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.style.use('ggplot')
data = np.random.randn(50)


print(plt.style.available)



['seaborn-bright', 'classic', 'seaborn-poster', 'seaborn-pastel', 'seaborn-darkgrid', 'Solarize_Light2', 'seaborn-whitegrid', 'tableau-colorblind10', 'seaborn-ticks', 'seaborn-muted', 'fast', 'seaborn-paper', 'fivethirtyeight', 'seaborn-dark-palette', '_classic_test', 'dark_background', 'seaborn', 'bmh', 'ggplot', 'seaborn-notebook', 'grayscale', 'seaborn-colorblind', 'seaborn-talk', 'seaborn-dark', 'seaborn-white', 'seaborn-deep']


This is a list of all the available styles built in to matplotlib. There are many!

### Assignment questions

After you've worked on some code, answer the following questions in this text block:

1.  Describe in a paragraph of text what you did and why, as if you were writing an email to somebody interested but nontechnical.

  *I used python to display a list of pre-made styles available for use within the matplotlib library. Matplotlib is a very useful library for displaying data processed in python in meaningful ways, such as graphs and charts. I can use any of the above listed styles as a shorthand for general formatting of the data I wish to present.*

2.  What was the most challenging part of what you did?

  *It took me a while to find the correct library in Github. The matplotlib is very large, and I'm as of yet unfamiliar with navigating github directories.*
 

3.  What was the most interesting thing you learned?

  *There are a lot of pre-made styles on matplotlib!*

4.  What area would you like to explore with more time?

  *I would like to experiment more and learn how to print a small graph displaying an example of each style.*




## Stretch goals and resources

Following are *optional* things for you to take a look at. Focus on the above assignment first, and make sure to commit and push your changes to GitHub (and since this is the first assignment of the sprint, open a PR as well).

- [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/)
- [scikit-learn documentation](http://scikit-learn.org/stable/documentation.html)
- [matplotlib documentation](https://matplotlib.org/contents.html)
- [Awesome Data Science](https://github.com/bulutyazilim/awesome-datascience) - a list of many types of DS resources

Stretch goals:

- Find and read blogs, walkthroughs, and other examples of people working through cool things with data science - and share with your classmates!
- Write a blog post (Medium is a popular place to publish) introducing yourself as somebody learning data science, and talking about what you've learned already and what you're excited to learn more about.